In [1]:
import requests 
from bs4 import BeautifulSoup
import re
import pandas as pd 
import pickle 
import string 
from dateutil import parser

In [2]:
def get_soup(start_url):
    response = requests.get(start_url)
    page = response.text
    return BeautifulSoup(page, "lxml")

In [13]:
from datetime import date, datetime, timedelta

def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

urls = []
for suffix in perdelta(date(2010, 1, 1), date(2021, 1, 1), timedelta(days=30)):
    prefix = 'http://www.billboard.com/charts/hot-100/'
    link = prefix + str(suffix)
    urls.append(link)

In [14]:
urls

['http://www.billboard.com/charts/hot-100/2010-01-01',
 'http://www.billboard.com/charts/hot-100/2010-01-31',
 'http://www.billboard.com/charts/hot-100/2010-03-02',
 'http://www.billboard.com/charts/hot-100/2010-04-01',
 'http://www.billboard.com/charts/hot-100/2010-05-01',
 'http://www.billboard.com/charts/hot-100/2010-05-31',
 'http://www.billboard.com/charts/hot-100/2010-06-30',
 'http://www.billboard.com/charts/hot-100/2010-07-30',
 'http://www.billboard.com/charts/hot-100/2010-08-29',
 'http://www.billboard.com/charts/hot-100/2010-09-28',
 'http://www.billboard.com/charts/hot-100/2010-10-28',
 'http://www.billboard.com/charts/hot-100/2010-11-27',
 'http://www.billboard.com/charts/hot-100/2010-12-27',
 'http://www.billboard.com/charts/hot-100/2011-01-26',
 'http://www.billboard.com/charts/hot-100/2011-02-25',
 'http://www.billboard.com/charts/hot-100/2011-03-27',
 'http://www.billboard.com/charts/hot-100/2011-04-26',
 'http://www.billboard.com/charts/hot-100/2011-05-26',
 'http://w

In [15]:
len(urls)

134

In [6]:
def get_artist(soup):
    artist = soup.find_all(class_="chart-element__information__song text--truncate color--primary")
    artists = []
    for item in artist:
        item = item.text.strip()
        artists.append(item)
    return artists

In [7]:
def get_track(soup):
    track = soup.find_all(class_="chart-element__information__artist text--truncate color--secondary")
    tracks = []
    for item in track:
        item = item.text.strip()
        tracks.append(item)
    return tracks



In [8]:
frames = []
billboard_dict = {}
for url in urls:
    soup = get_soup(url)
    artist = get_artist(soup)
    track = get_track(soup)
    dates = [parser.parse(url.split('/')[5])]*len(artist)
    df = pd.DataFrame({'artist': artist, 'track': track, 'publish_date': dates})
    frames.append(df)

In [9]:
df_merge = pd.concat(frames).reset_index(drop=True)

In [10]:
df_merge.to_csv('File1.csv')

In [11]:
df2 = pd.read_csv('File1.csv')
print(df2.shape)
df3 = df2.drop_duplicates(['track'], keep='first').reset_index(drop=True)

(1000, 4)


In [12]:
print(df3.shape)
df3.to_csv('File3.csv')

(288, 4)
